<a href="https://colab.research.google.com/github/amiable17/Deep_Learning/blob/master/10_%EB%AA%A8%EB%8D%B8%EC%84%A4%EA%B3%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **모두의 딥러닝**
## 넷째마당-딥러닝 기본기 다지기

* 참고 1)  
tensorflow 1. version 코드이기 때문에 현재 버전 2.0.0 에서 실행되지 않는 코드 있음  

>```
import tensorflow as tf  
tf.__version__   
tf.compat.v1.disable_eager_excute()  
sess = tf.compat.v1.Session()
```



* 참고 2)  
tf.global_variables_initializer()  
tf.Variable : 변수형 텐서. 초기값으로 설정할 값 입력이 필요함
```
a = tf.Variable(tf.ones(2,3))
```

***변수형 텐서 생성 시 입력한 초기값은 아직 해당 변수형 텐서의 값으로 할당되기 전 상태.  
따라서 초기화 연산자를 통해 초기값이 변수에 할당됨***  

> tf.global_variables_initializer()함수를 사용하면 그래프 내의 모든 변수의 초기화 연산 수행이 가능  
>tf.constant_initializer(a) 특정 변수를 초기화하는 연산 수행


* 참고 3)  
Learning Rate: cost 함수에서 변화하는 정도(=기울기의 크기)  
__learning rate가 너무 크면__  
  결과 : Overshooting  
  증상 : `NaN`이 나온다  
__learning rate가 너무 작으면__  
  결과 : 학습이 더디고, Local Minimum에 걸려서 제대로 된 최소값을 구하지 못하게 됩니다.  
  증상 : Cost 변화값이 너무 적게 변화한다.

### 10장 모델 설계하기
>   폐암 수술 환자의 생존율 예측하기

#### **딥러닝의 구조를 짜고 층 설정하기**: 입력층, 은닉층, 출력층  


(https://drive.google.com/uc?export=view&id=1qy5RXN5oJuFt8wHrg7Qf9uEydE8J6gh5)
<figure>
<center>
<img src='https://drive.google.com/open?id=1qy5RXN5oJuFt8wHrg7Qf9uEydE8J6gh5'/>
<figcaption>Image Caption</figcaption></center>
</figure>


> 1) Sequential(): 딥러닝의 구조를 한 층 한 층 쉽게 쌓아올리게 함  
> Dense(): 각 층이 제각각 어떤 특성을 가질지 옵션 설정.  
>> units: 노드의 수.  
input_dim: 입력 데이터로부터 몇 개의 값이 들어올지. feature의 수. 첫번째 은닉층에 적어줌으로써 입력층 + 은닉층의 역할.    
activation: 다음 층으로 어떻게 값을 넘길지 결정.  

> 2) model.compile(): 환경 설정 및 컴파일
>> loss: cost function  
optimizer: 최적화 방법 지정. 오차를 어떻게 줄여 나갈지 정하는 함수  
metrics: 모델 수행 결과를 나타냄.

> 3) model.fit(): 모델 실행
>> epochs:
batch_size: 샘플을 한번에 몇 개씩 처리할지.


#### **교차 엔트로피**
> 오차 함수 중 하나로 주로 *`분류 문제`*에서 많이 사용.  
출력 값에 로그를 취해서, 오차가 커지면 수렴 속도가 빨라지고 오차가 작아지면 속도가 감소.  
* 이항 분류: binary_crossentropy  
* 다항 분류: categorical_crossentropy


In [0]:
# 필요한 라이브러리 및 케라스 함수 불러오기
import numpy as np
import tensorflow as tf
from keras import models, layers

print("tensorflow version: ", tf.__version__)

tensorflow version:  1.15.0


Using TensorFlow backend.


In [0]:
# seed 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

* 일정한 결과값을 얻기 위해서는 numpy seed값과 tensorflow seed값을 모두 설정해줘야 함.  
* seed값을 지정해도 출력 값이 미세하게 다를 수 있는데, 이는 텐서플로를 구동하는 cuDNN 등의 내부 소프트웨어가 자체적으로 다른 랜덤 테이블을 생성하기 때문.
* 현재 이를 잡아줄 방법이 없기 때문에, 여러번 모델링 후 평균을 구하는 것이 가장 안전.

데이터 업로드 방법1: 로컬 데이터 colab에 업로드

In [0]:
from google.colab import files
uploaded = files.upload()

Saving ThoraricSurgery.csv to ThoraricSurgery (1).csv


데이터 업로드 방법2: 구글 드라이브에 마운트해서 데이터 업로드  
```
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
```

In [0]:
# 준비된 데이터 불러오기
import io
Data_set = np.loadtxt(io.StringIO(uploaded['ThoraricSurgery.csv'].decode('utf-8')), delimiter = ',')
print(Data_set.shape)
# 데이터: 폴란드 브로츠와프 의과대학에서 2013년 공개한 폐암 수술 환자의 수술 전 진단 데이터와 수술 후 생존 결과를 기록한 실제 의료 기록 데이터.
# 앞의 17개 정보는 종양의 유형, 폐활량, 호흡 곤란 여부, 고통 정도 등 17가지의 환자 상태. 18번째 정보는 수술 후 생존 결과(1=true,생존/0=false,사망)

(470, 18)


In [0]:
# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X = Data_set[:,0:17]
Y = Data_set[:,17]

In [0]:
# 딥러닝 구조 결정(모델 설정 & 컴파일)
# 두개의 층(= # model.add)을 가진 모델  
model = models.Sequential()
model.add(layers.Dense(30, input_dim=17, activation='relu'))     # 은닉층. 30개의 노드 & 17개의 feature가 들어옴. 
model.add(layers.Dense(1, activation='sigmoid'))                 # 출력층. units = 1. 1개의 값을 정해서 보내줘야함.

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])

신경망 모델 도식화(그림 삽입 필요, p.127)

In [0]:
# 딥러닝 실행
model.fit(X, Y, epochs = 30, batch_size = 10)               # epochs=30: 샘플이 처음부터 끝까지 30번 재사용될 때까지 실행을 반복, batch_size=10: 전체 데이터 샘플을 10개씩 끊어서 넣음.

# batch_size가 너무 크면 학습 속도가 느려지고, 너무 작으면 각 실행 값의 편차가 생겨 전체 결과값이 불안정해짐. 컴퓨터 메모리에 따라 설정 필요.

# 결과 출력
print("\n Accuracy: %.4f" %(model.evaluate(X,Y)[1]))




Epoch 1/30





470/470 [==============================] - 1s 1ms/step - loss: 0.6647 - acc: 0.3106
Epoch 2/30
470/470 [==============================] - 0s 107us/step - loss: 0.1488 - acc: 0.8511
Epoch 3/30
470/470 [==============================] - 0s 104us/step - loss: 0.1488 - acc: 0.8511
Epoch 4/30
470/470 [==============================] - 0s 100us/step - loss: 0.1488 - acc: 0.8511
Epoch 5/30
470/470 [==============================] - 0s 108us/step - loss: 0.1488 - acc: 0.8511
Epoch 6/30
470/470 [==============================] - 0s 113us/step - loss: 0.1487 - acc: 0.8511
Epoch 7/30
470/470 [==============================] - 0s 106us/step - loss: 0.1487 - acc: 0.8511
Epoch 8/30
470/470 [==============================] - 0s 114us/step - loss: 0.1487 - acc: 0.8511
Epoch 9/30
470/470 [==============================] - 0s 142us/step - loss: 0.1487 - acc: 0.8511
Epoch 10/30
470/470 [==============================] - 0s 107us/step - loss: 0.1486 - acc: 0.8511
Epoch 11/30
470/470 [==